# OceanOptics 2017
# Lab 5: Extracted Chlorophyll-a 


The purpose of this notebook is to calculate chlorophyll-a and pheophyton from our extracted chlorophyll-a samples that we have extracted. We will also propagate instrument uncertainty, compute the standard deviation of the replicates, and compare results bewrtwqeen students. Finally, we will calculate chlorophyll-a specific absorption coefficients for DRE samples. 

In [1]:
# This notebook was built in Catherine's root environment. (Sorry Don!) 
# authors:
# Catherine Kuhn
# Elena Terzić
# Brian Lamb
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import glob
import datetime
import pylab
import matplotlib as plt
import datetime as datetime  
import statsmodels.api as sm  
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\elena\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
%matplotlib inline

### Import the data

In [3]:
%cd /Users/ckuhn/Documents/oceanoptics/Labs/Lab05/ # set to your working directory

[WinError 3] The system cannot find the path specified: '/Users/ckuhn/Documents/oceanoptics/Labs/Lab05/ # set to your working directory'
C:\Users\elena\Dropbox\oceanoptics2017_students\Lab5_ap\Fluorometer


This .csv contains the chl a and pheo a already calculated from the spreadsheet that was given to use by our amazing TA Jordan

In [4]:
data = pd.read_csv('fluorometer_data.csv', delimiter='\t')

In [5]:
% pwd

'C:\\Users\\elena\\Dropbox\\oceanoptics2017_students\\Lab5_ap\\Fluorometer'

# Set your global variables

In [9]:
# Uncertainty terms:
    
# sigma K given in the lab

sigma_K = 0.03
    
# These numbers are the uncertainty associated with when the instrument read the chla and pheo during calibration. 
# We are ballparking these numbers

sigma_phi_chla = 3.
sigma_phi_pheo = 1.5

phi_chla = 500.  # these are the conc associated with the standards 
phi_pheo = 250.  # these are the conc associated with the standards
    
# These numbers are the uncertainty we observed when we were taking measurements. The initial chlorophyll and final 
# chlorophyll 

sigma_F_init  = 3.
sigma_F_final = 1.5
       
# These are the calibration coefficients that we pulled from the lab spreadsheet from the last calibration, which was 
# conducted on April 27, 2012
    
K = 0.36 
A = 1.88

# Volumes

sigma_v = 0.0005   # let's say we spilled 0.5 mL
sigma_V = 0.005   # let's say we spilled 5 mL

v = 0.005  # acetone
V = 0.289  # volume of water filtered for the sample  

### Add in your sample information

In [ ]:
data.head()

In [ ]:
#data.Fa.astype(np.float64)
#data.Fo.astype(np.float64)

Brian, this is how it works! WOW!

In [ ]:
varNo = 0 # change this to your sample number!

f_init = data['Fo'][varNo]
f_acid = data['Fa'][varNo]
chla_ugL = data['chl_ugL'][varNo]
pheo_ugL = data['phaeo_ugL'][varNo]

In [ ]:
print ('initial F = ', f_init)
print ('acid F =', f_acid)
print ('chla_ug/L =', chla_ugL)
print ('pheo_ug/L =', pheo_ugL)

Compute the uncertainty in your acid ratio (A)

In [ ]:
sigma_A = 1/A * np.sqrt(((sigma_phi_chla/phi_chla)**2) + ((sigma_phi_pheo/phi_pheo)**2))
print ('sigma_A=', sigma_A)

Compute the uncertainty in your sigma A/A-1

In [ ]:
sigma_AoverAminus1 = 1/ (A/(A-1)) * np.sqrt (((sigma_A/A)**2) + ((sigma_A/A)**2))
print(sigma_AoverAminus1)

Uncertainty in the flourescence readings

In [ ]:
sigma_Finit_minus_Ffinal = np.sqrt (((sigma_F_init)**2) + ((sigma_F_final)**2))
print(sigma_Finit_minus_Ffinal)

In [ ]:
sigma_vV = sigma_v/sigma_V
print(sigma_vV)

In [ ]:
sigma_chla = 1/chla_ugL * np.sqrt (((sigma_K/K)**2) + (sigma_AoverAminus1/ (A/(A-1))**2)
                                         + (sigma_Finit_minus_Ffinal/(f_init-f_acid)**2) 
                                         + ((sigma_vV/(v/V))**2))

## What the fuck is our uncertainty?

In [10]:
print(sigma_A)  # from calibration
print(sigma_K)   # more calibration error
print(sigma_vV) # from measuring water volume
print (sigma_Finit_minus_Ffinal) # uncertainty in our lab readings

print ('final error:', sigma_chla) # total problems

NameError: name 'sigma_A' is not defined

In [11]:
data['total_error_chla'] = sigma_chla

NameError: name 'sigma_chla' is not defined

In [ ]:
#errors = pd.Dataframe(sigma_A,sigma_K, sigma_vV, sigma_Finit_minus_Ffinal)
#my_list =  [sigma_A,sigma_K, sigma_vV, sigma_Finit_minus_Ffinal]

In [22]:
# One formula to rule them all 
sigma_A_ls = []
sigma_AoverAminus1_ls = []
sigma_Finit_minus_Ffinal_ls = []
sigma_vV_ls = []
sigma_chla_ls =[]

for row in range (len(data)):
    
# Extract measurements from sheet
    f_init = data['Fo'][row]
    f_init = np.asarray(f_init)
    print(f_init)
    f_acid = data['Fa'][row]
    f_acid = np.asarray(f_acid)
    chla_ugL = data['chl_ugL'][row]
    chla_ugL = np.asarray(chla_ugL)
    pheo_ugL = data['phaeo_ugL'][row]
    pheo_ugL = np.asarray(pheo_ugL)
    V        = data['V_sample_L'][row]
    V        = np.asarray(V)
    v        = data['v_acetone_L'][row].astype(np.float64)
    v   

# Compute sigmas
    sigma_A = 1./A * np.sqrt(((sigma_phi_chla/phi_chla)**2.) + ((sigma_phi_pheo/phi_pheo)**2.))
   
    sigma_AoverAminus1 = 1./ (A/(A-1)) * np.sqrt (((sigma_A/A)**2) + ((sigma_A/A)**2.))
    sigma_Finit_minus_Ffinal = np.sqrt (((sigma_F_init)**2.) + ((sigma_F_final)**2))
    sigma_vV = sigma_v/sigma_V
    sigma_chla = 1./chla_ugL * np.sqrt (((sigma_K/K)**2) + (sigma_AoverAminus1/ (A/(A-1.))**2.)
                                             + (sigma_Finit_minus_Ffinal/(f_init-f_acid)**2.) 
                                             + ((sigma_vV/(v/V))**2))
    sigma_A_ls.append(sigma_A)
    sigma_AoverAminus1_ls.append(sigma_AoverAminus1)
    sigma_Finit_minus_Ffinal_ls.append(sigma_Finit_minus_Ffinal)
    sigma_vV_ls.append(sigma_vV)
    sigma_chla_ls.append(sigma_chla)



590
573
536
638
609
594
541
340
589
569
563
623
538
606
590
582
594
381
591
573
575
577
546
531
536
516
546
580
604
154
667
633
603
652
547
723
654
733
599
707
611
384
645
690
691
486
436
454
404
449
428
541
424
426
461
443
442
487
495


In [19]:
sigma_AoverAminus1_ls

[0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312686012,
 0.0015892432312

In [17]:
for row in range (len(data)):
    print (row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
